In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import glob
import datetime
import numpy as np
import cv2
import time

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [2]:
import os

blocked_r_dir = 'dataset/blocked_r'
blocked_l_dir = 'dataset/blocked_l'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_r_dir)
    os.makedirs(blocked_l_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_r_button = widgets.Button(description='add blocked_r', button_style='danger', layout=button_layout)
blocked_l_button = widgets.Button(description='add blocked_l', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_r_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_r_dir)))
blocked_l_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_l_dir)))

###CONTROL BUTTON LAYOUT###
button_layout_ctrl = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout_ctrl)
forward_button = widgets.Button(description='forward', layout=button_layout_ctrl)
backward_button = widgets.Button(description='backward', layout=button_layout_ctrl)
left_button = widgets.Button(description='left', layout=button_layout_ctrl)
right_button = widgets.Button(description='right', layout=button_layout_ctrl)

middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
free_box = widgets.HBox([free_count, free_button])
block_r_box = widgets.HBox([blocked_r_count, blocked_r_button])
block_l_box = widgets.HBox([blocked_l_count, blocked_l_button])
fb_box = widgets.VBox([free_box, block_r_box, block_l_box])

display(widgets.HBox([fb_box, controls_box]))

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked_r():
    global blocked_r_dir, blocked_r_count
    save_snapshot(blocked_r_dir)
    blocked_r_count.value = len(os.listdir(blocked_r_dir))

def save_blocked_l():
    global blocked_l_dir, blocked_l_count
    save_snapshot(blocked_l_dir)
    blocked_l_count.value = len(os.listdir(blocked_l_dir))
    
###CONTROL BUTTON FUNCTION###
robot = Robot()
motor_speed_ratio = 0.3
time_interval = 0.1

def stop(change):
    robot.stop()

def step_forward(change):
    robot.forward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
def step_backward(change):
    robot.backward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
def step_left(change):
    robot.left(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def step_right(change):
    robot.right(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_r_button.on_click(lambda x: save_blocked_r())
blocked_l_button.on_click(lambda x: save_blocked_l())

####LINK BUTTONS TO ACTIONS####
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

In [5]:
display(image)
display(widgets.HBox([fb_box, controls_box]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…